In [ ]:
import re

In [ ]:
m = re.findall(r'\(.*\)', '00:000 (move_aside_across_tracks train_ddz_1 track_entry track_t_401)')
m

In [ ]:
file = "sas_plan"

with open(file, 'r') as f:
    lines = f.readlines()

parsed_lines = [re.findall(r'\(.*\)', line)[0][1:-1] for line in lines if any(x in line for x in ('track', 'train'))]
parsed_lines

In [ ]:
def extract_objects(lines):
    trains = set()
    tracks = set()
    for line in lines:
        elements = line.split(' ')
        train_names = set([t for t in elements if t.startswith('train_')])
        track_names = set([t for t in elements if t.startswith('track_')])
        trains = trains.union(train_names)
        tracks = tracks.union(track_names)
    return trains, tracks

trains, tracks = extract_objects(parsed_lines)
trains, tracks

In [61]:
def extract_movements(lines):
    movements = []
    for line in lines:
        if not line.startswith('move'):
            continue
        direction = 'aside' if 'aside' in line else 'bside'
        train = re.findall(r'train_\w+', line)[0]
        [track_from, track_to] = re.findall(r'track_\w+', line)
        movements.append((direction, train, track_from, track_to))
    return movements

movements = extract_movements(parsed_lines)
movements

[('aside', 'train_ddz_1', 'track_entry', 'track_t_401'),
 ('aside', 'train_ddz_2', 'track_entry', 'track_t_401'),
 ('aside', 'train_ddz_3', 'track_entry', 'track_t_402')]

In [53]:
def organize_movements(trains, tracks, movements):

    train_movements = {t:[] for t in trains}
    track_movements = {t:[] for t in tracks}

    for train in trains:
        for idx, movement in enumerate(movements):
            if train in movement:
                train_movements[train].append((idx+1, *movement))
            
    for track in tracks:
        for idx, movement in enumerate(movements):
            if track in movement:
                track_movements[track].append((idx+1, *movement))

    return train_movements, track_movements
train_moves, track_moves = organize_movements(trains, tracks, movements)


In [62]:
def no_overlap_pairs(train_movements, movements):
    
    no_overlaps = set()

    for train in train_movements.keys():
        if len(train_movements[train]) <= 1:
            continue
        for i in range(len(train_movements[train])-1):
            no_overlaps.add((train_movements[train][i][0], 
                             train_movements[train][i+1][0]))

    for i in range(len(movements)):
        for j in range(len(movements)):
            if i == j or j < i:
                continue
        
            from_1, to_1 = movements[i][2], movements[i][3]
            from_2, to_2 = movements[j][2], movements[j][3]

            if from_1 == to_2 or to_1 == to_2:
                no_overlaps.add((i, 
                                 j))        
        
        return no_overlaps

no_overlaps = no_overlap_pairs(train_moves, movements)
no_overlaps

{(0, 1)}

In [63]:
def ordered_action_pairs(track_movements, overlap_pairs):
    ordered_pairs = set()
    for track in track_movements.keys():
        if len(track_movements[track]) <= 1:
            continue
        for i in range(len(track_movements[track])-1):
            pair = (track_movements[track][i][0], 
                             track_movements[track][i+1][0])
            if pair not in overlap_pairs:
                ordered_pairs.add(pair)
    return ordered_pairs

ordered_pairs = ordered_action_pairs(track_moves, no_overlaps)
ordered_pairs

{(1, 2), (2, 3)}